In [78]:
import torch
import torch.nn.functional as F
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn
from torch.utils.data import DataLoader  
from torchvision import datasets
import torchvision.transforms as transforms  # 


In [79]:
from Chess_Data import ChessData

In [80]:
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
device

device(type='cuda')

In [92]:
class Test_Model(nn.Module):
    def __init__(self):
        super(Test_Model,self).__init__()
        self.conv1 = nn.Conv2d(7,128, kernel_size=3,padding=1)
        self.max = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(128,256, kernel_size=1)
        self.linear = nn.Sequential(nn.Linear(4*256, 512), nn.ReLU(), nn.Linear(512,128),
                                   nn.ReLU(), nn.Linear(128,64), nn.ReLU(), nn.Linear(64,2))
        

    def init_weights(self):
        torch.nn.init.normal_(self.conv1.weight,0, 1e-7)
        torch.nn.init.normal_(self.conv2.weight,0, 1e-7)
        torch.nn.init.normal_(self.conv3.weight,0, 1e-7)
        torch.nn.init.normal_(self.conv4.weight,0, 1e-6)
    
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.max(x)
        x = F.relu(self.conv2(x))
        x = self.max(x)
        x = x.reshape(x.shape[0], -1)
        x = self.linear(x)
        
        
        return x


        

In [93]:
trainset =  ChessData('training.csv')
trainloader= torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True, num_workers=0 )

In [94]:
def train(model,criterion,optimizer,num_epochs,trainloader):
    
    for epoch in range(num_epochs):
        running_loss=0.0
        
        for i , (  targets, data) in enumerate(trainloader):
            
            data = data.to(device)
            targets= targets.to(device)
            targets = targets.to(torch.int64)

            #forward prop
            scores = model(data).squeeze()
            

            loss = criterion(scores, targets)

            #backprop
            optimizer.zero_grad()
            loss.backward()

            #take step
            optimizer.step()
            losss= loss.item()
            
            running_loss+=losss
            
            
           
            if i %500==499:
                print('[%d,%5d] loss: %.3f' % (epoch+1 , i+1, running_loss/500))
                running_loss=0.0
            if i % 2000 == 1999: 
                print("Train accuracy: "+ str(check_accuracy(trainloader, model)*100))
               # print("Test accuracy: "+ str(check_accuracy(testloader, model)*100))
            

In [95]:
def check_accuracy(loader, model, full=False):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for  i ,( y, x) in enumerate(loader):
            if i >100 and not full:

                return num_correct/num_samples
            x = x.to(device)
            y = y.to(device)
            y =y.to(torch.int64)
           

            scores = model(x)
            
            
            _, predictions = scores.max(1)
            
            num_correct += (predictions == y).sum()
            
         
            
            
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples

In [96]:
my_model = Test_Model().to(device)

In [ ]:


batch_size = 16
learning_rate = 1e-8
num_epochs = 10
criterion3 = nn.CrossEntropyLoss()
optimizer3=optim.Adam(my_model.parameters(),lr=learning_rate)
#optimizer3 = optim.SGD(my_model.parameters(),lr=learning_rate,momentum=.9)
num_epochs=15
train(my_model,criterion3,optimizer3,num_epochs,trainloader)

#check_accuracy(trainloader, my_model)

[1,  500] loss: 0.541
[1, 1000] loss: 0.537
[1, 1500] loss: 0.541
[1, 2000] loss: 0.548
Train accuracy: tensor(71.4728, device='cuda:0')
[1, 2500] loss: 0.535
[1, 3000] loss: 0.542
[1, 3500] loss: 0.534
[1, 4000] loss: 0.538
Train accuracy: tensor(73.1436, device='cuda:0')
[1, 4500] loss: 0.540


In [77]:
torch.save(my_model.state_dict(), './Prototype2.0-Adam-SGD-1C5L0B-4.pth')


In [ ]:
test_model = Test_Model().to(device)
test_model.load_state_dict(torch.load('./Prototype1.0-Adam/SGD-2C3L0B-1.pth'))
check_accuracy(trainloader, test_model, full=True)

In [ ]:
'''./Prototype2.0-Adam-SGD-1C5L0B-2.pth
class Test_Model(nn.Module):
    def __init__(self):
        super(Test_Model,self).__init__()
        self.conv1 = nn.Conv2d(7,128, kernel_size=3,stride=1)
        self.max = nn.MaxPool2d(2,2)
        self.linear = nn.Sequential(nn.Linear(1152, 512), nn.ReLU(), nn.Linear(512,512), nn.ReLU(), nn.Linear(512,128),
                                   nn.ReLU(), nn.Linear(128,64), nn.ReLU(), nn.Linear(64,2))
        

    def init_weights(self):
        torch.nn.init.normal_(self.conv1.weight,0, 1e-7)
        torch.nn.init.normal_(self.conv2.weight,0, 1e-7)
        torch.nn.init.normal_(self.conv3.weight,0, 1e-7)
        torch.nn.init.normal_(self.conv4.weight,0, 1e-6)
    
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.max(x)
        x = x.reshape(x.shape[0], -1)
        print(x.shape)
        x = self.linear(x)
        
        
        return x
'''
